In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [2]:
# read in file
nRowsRead = 284807
df1 = pd.read_csv('C:\\Users\\ajens\\OneDrive\\Documents\\Personal\\A.Jensen MDS\\creditcard.csv\\creditcard.csv', delimiter=',', nrows= nRowsRead)
df1.dataframeName = 'Fraud Detection'

In [41]:
Class = pd.DataFrame(df1['Class'],columns=['Class'])

In [42]:
df1.drop(['Time', 'Class'], axis=1,inplace=True)

In [44]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [45]:
pipe = make_pipeline(
    MinMaxScaler(),
    GradientBoostingClassifier(random_state=42,
                             n_estimators=100,
                             verbose=False)
)
X_train, X_test, y_train, y_test = train_test_split(df1, Class, test_size=0.3, random_state=42)
pipe.fit(X_train, y_train).score(X_test, y_test)

0.9985838512224524

In [63]:
y_pred = pipe.predict(X_test)
print(pipe,"\n\nAccuracy: ",accuracy_score(y_test,ypred)*100)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(random_state=42, verbose=False))]) 

Accuracy:  99.85838512224524


The main problem of imbalanced data sets lies on the fact that they are often associated with a user preference bias towards the performance on cases that are poorly represented in the available data sample. We will see a better representation of our metrics after we balance our data.

In [55]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print(classification_report(y_test, y_pred))

[[85305     2]
 [  119    17]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.89      0.12      0.22       136

    accuracy                           1.00     85443
   macro avg       0.95      0.56      0.61     85443
weighted avg       1.00      1.00      1.00     85443



In [56]:
# Creating a function to report confusion metrics
def confusion_metrics (conf_matrix):
# save confusion matrix and slice into four pieces
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    print('True Positives:', TP)
    print('True Negatives:', TN)
    print('False Positives:', FP)
    print('False Negatives:', FN)
    
    # calculate accuracy
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN)) 
    # calculate the specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))
    # calculate f_1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    print('-'*40)
    print(f'Accuracy: {round(conf_accuracy * 100.0,4)}%') 
    print(f'Sensitivity: {round(conf_sensitivity * 100.0,4)}%') 
    print(f'Specificity: {round(conf_specificity * 100.0,4)}%') 
    print(f'Precision: {round(conf_precision * 100.0,4)}%')
    print(f'f_1 Score: {round(conf_f1 * 100.0,4)}%')

In [57]:
confusion_metrics(confusion_matrix)

True Positives: 17
True Negatives: 85305
False Positives: 2
False Negatives: 119
----------------------------------------
Accuracy: 99.8584%
Sensitivity: 12.5%
Specificity: 99.9977%
Precision: 99.9977%
f_1 Score: 22.2222%


Sensitivity (aka Recall) means out of all actual Positives, how many did we predict as Positive.

Specificity (aka Selectivity or True Negative Rate, TNR) means out of all actual Negatives, how many did we predict as Negative.

Precision (aka Positive Predictive Value, PPV) means “out of all predicted Positive cases, how many were actually Positive.

F1 Score is the harmonic, or weighted average of Precision and Sensitivity.

In [67]:
# cross validation score
scores= cross_val_score(pipe, df1, Class, cv= 10)
print(scores)
print(f'\nAverage Cross Validation score= {np.mean(scores)}')

[0.99831467 0.99884133 0.99961378 0.99852533 0.99831467 0.99856044
 0.99957867 0.99852528 0.99863062 0.99870084]

Average Cross Validation score= 0.9987605626794759


Our data ran through the pipeline with cross validation appears to perform slightly better, however, the improved performance is insignificant. 